## Creación de una base de datos unificada.

![Esquema de la base de datos](../img/database_schema.png)

In [1]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

Creo el motor de base de datos.

In [2]:
engine = create_engine('sqlite:///../database.db', echo=True)

In [3]:
def crear_tabla(nombre, data: pd.DataFrame):
    try:
        data.to_sql(name=nombre, con=engine, index=False, if_exists='replace')
        print(f'Tabla "{nombre}" creada con exito.')
    except:
        raise "Error al crear la tabla."

# Tablas de hecho.

### Tabla `clima`.

In [27]:
df = pd.read_parquet('../data/datos_climaticos.parquet')

In [32]:
df = df.loc[df.fecha > "2017-12-31"].reset_index(drop=True)

In [38]:
df['anio'] = df.fecha.dt.year
df['semana'] = df.fecha.dt.isocalendar().week

df.sample(10)

,fecha,id_estacion,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa,anio,semana
9387,2023-02-26,A872812,0.0,18.1,33.0,24.62430,78.0,21.119760,25.164120,21.851280,8.858451,70.000000,2023,8
50507,2023-05-11,A872956,0.0,8.8,27.3,16.94028,70.0,12.062300,14.153020,13.927020,8.248169,77.000000,2023,19
69090,2019-09-25,NH0098,0.0,6.1,28.2,20.80000,49.0,8.479503,11.217895,21.510073,10.200000,83.000000,2019,39
37186,2019-05-10,A872950,0.0,16.5,22.2,19.42013,100.0,18.349300,21.206830,3.539220,4.553577,42.500000,2019,19
30593,2021-05-30,A872916,0.0,6.6,19.9,12.51250,71.0,6.336804,9.622926,13.550700,8.779555,84.000000,2021,21
21314,2024-06-05,A872899,0.0,10.9,29.8,18.18264,81.0,16.429650,19.088490,10.566460,5.945106,56.000000,2024,23
72150,2022-12-02,NH0114,0.0,14.4,32.1,23.30000,50.0,17.481130,14.308789,29.200000,11.900000,95.275894,2022,48
87466,2022-03-05,NH0421,0.0,10.4,25.9,18.15000,63.0,10.975442,13.106164,22.056938,10.900000,85.490000,2022,9
72862,2018-05-12,NH0216,0.0,4.1,18.0,11.05000,68.0,3.690000,7.900000,12.300000,9.700000,96.000000,2018,19
88110,2023-12-09,NH0421,0.0,11.6,30.7,21.15000,56.0,13.461910,15.607573,19.888226,7.200000,48.927937,2023,49


In [47]:
df.drop('fecha', axis=1).groupby(by=['id_estacion', 'anio', 'semana']).mean().reset_index()

,id_estacion,anio,semana,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
0,A872801,2018,1,0.073696,14.925000,30.975000,23.480816,50.500000,12.928104,15.256275,20.920133,8.970463,62.562500
1,A872801,2018,2,0.071282,20.857143,32.471429,26.344840,58.142857,17.314917,20.053073,19.498533,7.859546,55.142857
2,A872801,2018,3,0.000000,17.800000,32.828571,25.153671,56.428571,16.678904,19.247406,23.093349,8.924465,63.428571
3,A872801,2018,4,0.065213,18.557143,31.300000,24.316767,58.000000,16.353461,19.074726,22.105954,8.489425,61.142857
4,A872801,2018,5,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,18.700631,24.891456,10.331836,75.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16442,NH0550,2025,33,0.000000,7.042857,24.578571,15.639286,57.000000,8.708845,10.412464,12.996384,6.671429,60.365714
16443,NH0550,2025,34,0.000000,6.371429,26.150000,15.714286,53.571429,8.432129,10.056680,16.375538,8.792857,78.218571
16444,NH0550,2025,35,0.026685,4.814286,27.871429,16.764286,43.000000,8.629537,8.984962,17.522917,9.157143,80.257143
16445,NH0550,2025,36,0.020187,4.542857,22.800000,13.671428,54.809524,7.155052,8.912380,13.904473,6.085714,52.485714


In [12]:
crear_tabla('clima', df)

2025-10-23 18:04:09,132 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:04:09,149 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-23 18:04:09,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,154 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-23 18:04:09,155 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,157 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-23 18:04:09,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,160 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-23 18:04:09,162 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,164 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("clima")
2025-10-23 18:04:09,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:0

### Tabla `contagios`

In [6]:
df_contagios = pd.read_csv('../data/dengue-final.csv')
df_contagios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68126 entries, 0 to 68125
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_uta                   68126 non-null  int64  
 1   departamento_nombre      68126 non-null  object 
 2   provincia_nombre         68126 non-null  object 
 3   ano                      68126 non-null  int64  
 4   semanas_epidemiologicas  68126 non-null  float64
 5   grupo_edad_id            68126 non-null  int64  
 6   grupo_edad_desc          68126 non-null  object 
 7   cantidad_casos           68126 non-null  float64
 8   poblacion                68126 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 4.7+ MB


In [7]:
df_contagios.head()

,id_uta,departamento_nombre,provincia_nombre,ano,semanas_epidemiologicas,grupo_edad_id,grupo_edad_desc,cantidad_casos,poblacion
0,2003,comuna 3,ciudad de buenos aires,2018,15.0,6,de 15 a 19 anos,1.0,192945
1,2004,comuna 4,ciudad de buenos aires,2018,6.0,6,de 15 a 19 anos,1.0,239279
2,6091,berazategui,buenos aires,2018,11.0,7,de 20 a 24 anos,1.0,358262
3,6091,berazategui,buenos aires,2018,14.0,9,de 35 a 44 anos,2.0,358262
4,6091,berazategui,buenos aires,2018,19.0,11,mayores de 65 anos,1.0,358262


In [ ]:
crear_tabla('cotagios', df_contagios)

2025-10-23 18:04:42,245 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:04:42,254 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cotagios")
2025-10-23 18:04:42,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:42,256 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cotagios")
2025-10-23 18:04:42,258 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:42,264 INFO sqlalchemy.engine.Engine 
CREATE TABLE cotagios (
	id_uta BIGINT, 
	departamento_nombre TEXT, 
	provincia_nombre TEXT, 
	ano BIGINT, 
	semanas_epidemiologicas FLOAT, 
	grupo_edad_id BIGINT, 
	grupo_edad_desc TEXT, 
	cantidad_casos FLOAT, 
	poblacion BIGINT
)


2025-10-23 18:04:42,265 INFO sqlalchemy.engine.Engine [no key 0.00115s] ()
2025-10-23 18:04:42,981 INFO sqlalchemy.engine.Engine INSERT INTO cotagios (id_uta, departamento_nombre, provincia_nombre, ano, semanas_epidemiologicas, grupo_edad_id, grupo_edad_desc, cantidad_casos, poblacion) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-

# Tablas de dimensiones.

### Tabla `calendario`.

In [25]:
df3 = pd.read_csv('../data/calendario.csv')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24004 entries, 0 to 24003
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fecha            24004 non-null  object
 1   anio             24004 non-null  int64 
 2   mes              24004 non-null  int64 
 3   dia              24004 non-null  int64 
 4   trimestre        24004 non-null  int64 
 5   semestre         24004 non-null  int64 
 6   quincena         24004 non-null  int64 
 7   semanaMes        24004 non-null  int64 
 8   semana           24004 non-null  int64 
 9   diaSemana        24004 non-null  object
 10  diaNumeroSemana  24004 non-null  int64 
 11  bisiesto         24004 non-null  bool  
dtypes: bool(1), int64(9), object(2)
memory usage: 2.0+ MB


In [26]:
df3

,fecha,anio,mes,dia,trimestre,semestre,quincena,semanaMes,semana,diaSemana,diaNumeroSemana,bisiesto
0,1960-01-01,1960,1,1,1,1,1,1,53,Viernes,4,True
1,1960-01-02,1960,1,2,1,1,1,1,53,Sábado,5,True
2,1960-01-03,1960,1,3,1,1,1,1,53,Domingo,6,True
3,1960-01-04,1960,1,4,1,1,1,1,1,Lunes,0,True
4,1960-01-05,1960,1,5,1,1,1,1,1,Martes,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
23999,2025-09-15,2025,9,15,3,2,1,3,38,Lunes,0,False
24000,2025-09-16,2025,9,16,3,2,2,3,38,Martes,1,False
24001,2025-09-17,2025,9,17,3,2,2,3,38,Miércoles,2,False
24002,2025-09-18,2025,9,18,3,2,2,3,38,Jueves,3,False


In [15]:
crear_tabla('calendario', df3)

2025-10-23 18:05:49,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:05:49,806 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calendario")
2025-10-23 18:05:49,807 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:05:49,809 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("calendario")
2025-10-23 18:05:49,810 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:05:49,816 INFO sqlalchemy.engine.Engine 
CREATE TABLE calendario (
	fecha TEXT, 
	anio BIGINT, 
	mes BIGINT, 
	dia BIGINT, 
	trimestre BIGINT, 
	semestre BIGINT, 
	quincena BIGINT, 
	"semanaMes" BIGINT, 
	semana BIGINT, 
	"diaSemana" TEXT, 
	"diaNumeroSemana" BIGINT, 
	bisiesto BOOLEAN
)


2025-10-23 18:05:49,818 INFO sqlalchemy.engine.Engine [no key 0.00272s] ()
2025-10-23 18:05:50,162 INFO sqlalchemy.engine.Engine INSERT INTO calendario (fecha, anio, mes, dia, trimestre, semestre, quincena, "semanaMes", semana, "diaSemana", "diaNumeroSemana", bisiesto) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 

### Tabla `ubicacion`

In [4]:
df_estaciones = pd.read_csv('../data/estaciones-meteorologicas-inta.csv')

In [8]:
lista_provincias = df_contagios.provincia_nombre.apply(lambda x: ' '.join(p.capitalize() for p in x.split(' '))).unique()
print(lista_provincias)

['Ciudad De Buenos Aires' 'Buenos Aires' 'Mendoza' 'Santa Fe' 'Rio Negro'
 'Cordoba' 'Corrientes' 'Formosa' 'Chaco' 'La Rioja' 'San Luis'
 'Entre Rios' 'Misiones' 'Salta' 'Santiago Del Estero' 'Tucuman' 'Jujuy'
 'Catamarca' 'Neuquen' 'Chubut' 'La Pampa' 'San Juan' 'Santa Cruz'
 'Tierra Del Fuego Antartida E Islas Del Atlantico Sur']


In [9]:
df_provincias = pd.DataFrame({'provincia': lista_provincias})
df_provincias

,provincia
0,Ciudad De Buenos Aires
1,Buenos Aires
2,Mendoza
3,Santa Fe
4,Rio Negro
5,Cordoba
6,Corrientes
7,Formosa
8,Chaco
9,La Rioja


In [10]:
mapeo_renombrar = {
    'Cordoba': 'Córdoba', 
    'Tierra Del Fuego Antartida E Islas Del Atlantico Sur': 'Tierra del Fuego',
    'Ciudad De Buenos Aires': 'CABA',
    'Tucuman': 'Tucumán',
    'Neuquen': 'Neuquén',
    'Entre Rios': 'Entre Ríos',
    'Rio Negro': 'Río Negro'
}

df_provincias = df_provincias.replace(mapeo_renombrar).sort_values('provincia').reset_index(drop=True)
df_provincias

,provincia
0,Buenos Aires
1,CABA
2,Catamarca
3,Chaco
4,Chubut
5,Corrientes
6,Córdoba
7,Entre Ríos
8,Formosa
9,Jujuy


In [11]:
df_provincias['id_provincia'] = range(len(df_provincias))
df_provincias

,provincia,id_provincia
0,Buenos Aires,0
1,CABA,1
2,Catamarca,2
3,Chaco,3
4,Chubut,4
5,Corrientes,5
6,Córdoba,6
7,Entre Ríos,7
8,Formosa,8
9,Jujuy,9


In [12]:
crear_tabla('provincias', df_provincias)

2025-10-24 08:45:02,543 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2025-10-24 08:45:02,549 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("provincias")
2025-10-24 08:45:02,549 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 08:45:02,551 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("provincias")
2025-10-24 08:45:02,552 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 08:45:02,553 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 08:45:02,554 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 08:45:02,556 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-24 08:45:02,557 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 08:45:02,558 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("provincias")
2025-10-24 08:45:02,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-24 08:45:02,559 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SE

### Tabla `localidades`

In [16]:
df_localidades = df_estaciones[['Localidad', 'Provincia', 'Latitud', 'Longitud']]
df_localidades

,Localidad,Provincia,Latitud,Longitud
0,25 de Mayo,Buenos Aires,-35.48,-60.13
1,Hurlingham,Buenos Aires,-85.00,-55.00
2,Hurlingham,Buenos Aires,-85.00,-60.00
3,loc_prueba,Sin asignar,-84.00,-70.00
4,loc_prueba,Sin asignar,-84.00,-65.00
...,...,...,...,...
164,Manfredi,Córdoba,-31.94,-65.22
165,Villa Mercedes,San Luis,-33.66,-65.41
166,Villa Paranacito,Entre Rios,-33.71,-58.65
167,Villa Ramallo,Buenos Aires,-33.52,-60.11


In [26]:
condiciones_todrop = (df_localidades.Localidad == 'loc_prueba') | (df_localidades.Provincia == 'Sin asignar')
df_localidades.loc[condiciones_todrop]

,Localidad,Provincia,Latitud,Longitud
3,loc_prueba,Sin asignar,-84.00,-70.00
4,loc_prueba,Sin asignar,-84.00,-65.00
5,loc_prueba,Sin asignar,-84.00,-60.00
6,loc_prueba,Sin asignar,-84.00,-55.00
7,loc_prueba,Sin asignar,-84.00,-50.00
8,loc_prueba,Sin asignar,-84.00,-45.00
9,loc_prueba,Sin asignar,-84.00,-40.00
29,A definir,Sin asignar,-36.86,-62.45


In [30]:
df_localidades = df_localidades.drop(df_localidades.loc[condiciones_todrop].index)\
    .sort_values(by=['Provincia', 'Localidad'])\
    .reset_index(drop=True)

df_localidades

,Localidad,Provincia,Latitud,Longitud
0,25 de Mayo,Buenos Aires,-35.48,-60.13
1,Arrecifes,Buenos Aires,-34.05,-60.14
2,Balcarce,Buenos Aires,-37.76,-58.30
3,Balcarce,Buenos Aires,-37.75,-58.30
4,Bordenave,Buenos Aires,-37.75,-63.08
...,...,...,...,...
156,Vaca Huañuna,Santiago del Estero,-27.47,-63.47
157,San Sebastian,Tierra del Fuego,-52.89,-68.45
158,Famaillá,Tucuman,-27.02,-65.38
159,Famaillá,Tucuman,-27.05,-65.42


In [31]:
df_localidades['id_localidad'] = range(len(df_localidades))
df_localidades

,Localidad,Provincia,Latitud,Longitud,id_localidad
0,25 de Mayo,Buenos Aires,-35.48,-60.13,0
1,Arrecifes,Buenos Aires,-34.05,-60.14,1
2,Balcarce,Buenos Aires,-37.76,-58.30,2
3,Balcarce,Buenos Aires,-37.75,-58.30,3
4,Bordenave,Buenos Aires,-37.75,-63.08,4
...,...,...,...,...,...
156,Vaca Huañuna,Santiago del Estero,-27.47,-63.47,156
157,San Sebastian,Tierra del Fuego,-52.89,-68.45,157
158,Famaillá,Tucuman,-27.02,-65.38,158
159,Famaillá,Tucuman,-27.05,-65.42,159


In [32]:
df_localidades.columns = list(col.lower().strip() for col in df_localidades.columns)
df_localidades

,localidad,provincia,latitud,longitud,id_localidad
0,25 de Mayo,Buenos Aires,-35.48,-60.13,0
1,Arrecifes,Buenos Aires,-34.05,-60.14,1
2,Balcarce,Buenos Aires,-37.76,-58.30,2
3,Balcarce,Buenos Aires,-37.75,-58.30,3
4,Bordenave,Buenos Aires,-37.75,-63.08,4
...,...,...,...,...,...
156,Vaca Huañuna,Santiago del Estero,-27.47,-63.47,156
157,San Sebastian,Tierra del Fuego,-52.89,-68.45,157
158,Famaillá,Tucuman,-27.02,-65.38,158
159,Famaillá,Tucuman,-27.05,-65.42,159


Reordeno las columnas.

In [33]:
df_localidades = df_localidades[[
    'id_localidad', 'provincia', 'localidad', 'latitud', 'longitud'
]]

df_localidades

,id_localidad,provincia,localidad,latitud,longitud
0,0,Buenos Aires,25 de Mayo,-35.48,-60.13
1,1,Buenos Aires,Arrecifes,-34.05,-60.14
2,2,Buenos Aires,Balcarce,-37.76,-58.30
3,3,Buenos Aires,Balcarce,-37.75,-58.30
4,4,Buenos Aires,Bordenave,-37.75,-63.08
...,...,...,...,...,...
156,156,Santiago del Estero,Vaca Huañuna,-27.47,-63.47
157,157,Tierra del Fuego,San Sebastian,-52.89,-68.45
158,158,Tucuman,Famaillá,-27.02,-65.38
159,159,Tucuman,Famaillá,-27.05,-65.42


In [34]:
df_departamentos = pd.read_csv('../data/departamentos.csv')
df_departamentos

,429: Too Many Requests
For more on scraping GitHub and how it may affect your rights,please review our Terms of Service (https://d...


## Chequeo de los datos en *`database.db`*

In [5]:
conn = sqlite3.connect('base_datos.db')
cursor = conn.cursor()

In [7]:
cursor.execute('SELECT * FROM clima LIMIT 5;').fetchall()

[('2017-09-19 00:00:00.000000',
  'A872801',
  0.0,
  10.9,
  23.0,
  17.3875,
  74.0,
  14.14647,
  16.19621,
  19.46478,
  9.991712,
  83.0),
 ('2017-09-20 00:00:00.000000',
  'A872801',
  0.4957418023473161,
  15.9,
  27.1,
  20.12014,
  70.0,
  15.29874,
  17.66667,
  9.588419,
  2.886585,
  24.0),
 ('2017-09-21 00:00:00.000000',
  'A872801',
  0.5260351212713814,
  9.4,
  16.6,
  11.65834,
  98.0,
  9.864174,
  12.19869,
  1.33866,
  6.4341365,
  53.5),
 ('2017-09-22 00:00:00.000000',
  'A872801',
  0.0,
  4.7,
  19.6,
  11.72778,
  75.0,
  7.075937,
  10.12698,
  19.80138,
  9.981688,
  83.0),
 ('2017-09-23 00:00:00.000000',
  'A872801',
  0.0,
  5.0,
  23.8,
  13.49166,
  65.0,
  8.476982,
  11.17051,
  19.09344,
  9.404964,
  78.0)]